In [ ]:
!pip install "gspread==6.1.3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.1.4
    Uninstalling gspread-6.1.4:
      Successfully uninstalled gspread-6.1.4


In [ ]:
import gspread
from google.oauth2.service_account import Credentials

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    '/content/my-project-response-29749ee50e47.json',
    scopes=scopes
)

gc = gspread.authorize(credentials)
sh = gc.open_by_key("1Ht23dgNixm40ZF8etrGH1584oaObYwUekske9G06IKk")

In [ ]:
import pandas as pd

worksheet = sh.sheet1 # or sh.get_worksheet(0) for the first sheet

# Get all values from the worksheet
data = worksheet.get_all_values()

# Convert to DataFrame
df_sheets = pd.DataFrame(data[1:], columns=data[0])  # First row is header

df_sheets.head()

,subject,body,answer,type,queue,priority,language,business_type,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9
0,Problema crítico del servidor requiere atenció...,Es necesaria una investigación inmediata sobre...,Estamos investigando urgentemente el problema ...,Incident,Technical Support,high,es,IT Services,Urgent Issue,Service Disruption,Incident Report,Service Recovery,System Maintenance,,,,
1,Anfrage zur Verfügbarkeit des Dell XPS 13 9310,"Sehr geehrter Kundenservice,\n\nich hoffe, die...","Sehr geehrter <name>,\n\nvielen Dank, dass Sie...",Request,Customer Service,low,de,Tech Online Store,Sales Inquiry,Product Support,Customer Service,Order Issue,Returns and Exchanges,,,,
2,Erro na Autocompletação de Código do IntelliJ ...,"Prezado Suporte ao Cliente <name>,\n\nEstou es...","Prezado <name>,\n\nObrigado por entrar em cont...",Incident,Technical Support,high,pt,IT Services,Technical Support,Software Bug,Problem Resolution,Urgent Issue,IT Support,,,,
3,Urgent Assistance Required: AWS Service,"Dear IT Services Support Team, \n\nI am reachi...","Dear <name>,\n\nThank you for reaching out reg...",Request,IT Support,high,en,IT Services,IT Support,Urgent Issue,Service Notification,Cloud Services,Problem Resolution,Technical Guidance,Performance Tuning,,
4,Problème d'affichage de MacBook Air,Cher équipe de support du magasin en ligne Tec...,"Cher <name>,\n\nMerci de nous avoir contactés ...",Incident,Product Support,low,fr,Tech Online Store,Technical Support,Product Support,Hardware Failure,Service Recovery,Routine Request,,,,


In [ ]:
df_sheets.shape

(4000, 17)

In [ ]:
df_sheets.loc[df_sheets["priority"].eq("high"), :].head()

,subject,body,answer,type,queue,priority,language,business_type,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9
0,Problema crítico del servidor requiere atenció...,Es necesaria una investigación inmediata sobre...,Estamos investigando urgentemente el problema ...,Incident,Technical Support,high,es,IT Services,Urgent Issue,Service Disruption,Incident Report,Service Recovery,System Maintenance,,,,
2,Erro na Autocompletação de Código do IntelliJ ...,"Prezado Suporte ao Cliente <name>,\n\nEstou es...","Prezado <name>,\n\nObrigado por entrar em cont...",Incident,Technical Support,high,pt,IT Services,Technical Support,Software Bug,Problem Resolution,Urgent Issue,IT Support,,,,
3,Urgent Assistance Required: AWS Service,"Dear IT Services Support Team, \n\nI am reachi...","Dear <name>,\n\nThank you for reaching out reg...",Request,IT Support,high,en,IT Services,IT Support,Urgent Issue,Service Notification,Cloud Services,Problem Resolution,Technical Guidance,Performance Tuning,,
5,Urgent: Issue with Zoom Screen Sharing Feature,"Dear Customer Support,\n\nI am experiencing a ...","Dear <name>,\n\nThank you for reaching out. We...",Incident,Technical Support,high,en,IT Services,Technical Support,Urgent Issue,Service Disruption,Problem Resolution,Software Bug,,,,
7,Service outage resolution requested for ongoin...,"Dear IT Services Customer Support,\n\nI am wri...","Dear <name>,\n\nThank you for reaching out to ...",Change,Service Outages and Maintenance,high,en,IT Services,Service Outage,IT Support,Problem Resolution,Technical Support,Service Recovery,System Maintenance,Performance Tuning,Urgent Issue,


In [ ]:
df_sheets.loc[df_sheets["priority"].eq("high"), :].shape[0] / df_sheets.shape[0]

0.41225

In [ ]:
# prompt: combine all tags columns from 1 to 9, and get value counts for most occuring words in the column

# Combine all tags columns from 1 to 9
#tags_columns = ['tags_' + str(i) for i in range(1, 10)]
tags_columns = [f"tag_{i}" for i in range(1,10)] #include all tags upto 9
df_sheets['all_tags'] = df_sheets[tags_columns].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

# Get value counts for most occurring words
from collections import Counter

# Combine all tags into a single string
all_tags = ' '.join(df_sheets['all_tags']).lower()

# Split the string into words
words = all_tags.split()

# Count the occurrences of each word
word_counts = Counter(words)

# Get the most common words
most_common_words = word_counts.most_common(50) # Get top 10, change as needed

most_common_words

[('support', 6059),
 ('technical', 4186),
 ('issue', 3105),
 ('service', 3078),
 ('problem', 2837),
 ('resolution', 2836),
 ('product', 1912),
 ('urgent', 1766),
 ('customer', 1116),
 ('software', 1045),
 ('bug', 1041),
 ('it', 1026),
 ('guidance', 1023),
 ('disruption', 895),
 ('failure', 870),
 ('hardware', 814),
 ('inquiry', 762),
 ('performance', 736),
 ('tuning', 735),
 ('recovery', 656),
 ('system', 655),
 ('account', 597),
 ('general', 594),
 ('assistance', 571),
 ('network', 547),
 ('request', 479),
 ('maintenance', 442),
 ('incident', 398),
 ('report', 398),
 ('billing', 380),
 ('and', 320),
 ('exchanges', 320),
 ('returns', 318),
 ('notification', 306),
 ('order', 226),
 ('payment', 215),
 ('processing', 215),
 ('crash', 213),
 ('warranty', 211),
 ('claim', 211),
 ('refund', 177),
 ('sales', 148),
 ('outage', 140),
 ('feature', 134),
 ('login', 99),
 ('documentation', 79),
 ('services', 78),
 ('cloud', 76),
 ('replacement', 62),
 ('critical', 55)]

In [ ]:
#issue,problem,urgent,disruption,failure,incident,crash,refund,outage,critical

In [ ]:
# def escalateit(incoming_issues):
#   if incoming_issues["priority"] == "high":
#     tags_combine = " ".join([incoming_issues[f"tag_{i+1}"] for i in range(0,9)])
#     for key in ["issue","problem","urgent","disruption","failure","incident","crash","refund","outage","critical"]:
#       if key.lower() in tags_combine.lower():
#         return True
#   return False


In [ ]:
def escalateit(incoming_issues):
    tags_combine = " ".join([incoming_issues.get(f"tag_{i+1}", "") for i in range(9)])

    # Check for high-priority and critical keywords
    critical_keywords = ["issue", "problem", "urgent", "disruption", "failure",
                         "incident", "crash", "refund", "outage", "critical"]
    if incoming_issues["priority"] == "high" and any(key.lower() in tags_combine.lower() for key in critical_keywords):
        return True

    # Escalate if specific tags are present
    specific_tags = ["security", "data breach", "compliance"]
    if any(tag.lower() in tags_combine.lower() for tag in specific_tags):
        return True

    return False

In [ ]:
# prompt: call the above function to check issues escalated

# Assuming 'incoming_issues' is a dictionary representing a row from your dataframe
# Example usage (replace with your actual data)
# Assuming you have a dictionary called 'sample_issue' with data similar to the dataframe row.

sample_issue = df_sheets.iloc[0].to_dict()  # Get the first row as a dictionary
is_escalated = escalateit(sample_issue)
print(f"Is the issue escalated? {is_escalated}")

# Example usage with another row
sample_issue_2 = df_sheets.iloc[1].to_dict()  # Get the second row as a dictionary
is_escalated = escalateit(sample_issue_2)
print(f"Is the issue escalated? {is_escalated}")

Is the issue escalated? True
Is the issue escalated? False


In [ ]:
sample_issue

{'subject': 'Problema crítico del servidor requiere atención inmediata',
 'body': 'Es necesaria una investigación inmediata sobre la interrupción en el servicio de gestión de AWS que está impactando funciones comerciales esenciales.',
 'answer': 'Estamos investigando urgentemente el problema con el servicio de gestión de AWS. Proporcionaremos actualizaciones sobre nuestro progreso.',
 'type': 'Incident',
 'queue': 'Technical Support',
 'priority': 'high',
 'language': 'es',
 'business_type': 'IT Services',
 'tag_1': 'Urgent Issue',
 'tag_2': 'Service Disruption',
 'tag_3': 'Incident Report',
 'tag_4': 'Service Recovery',
 'tag_5': 'System Maintenance',
 'tag_6': '',
 'tag_7': '',
 'tag_8': '',
 'tag_9': '',
 'all_tags': 'Urgent Issue Service Disruption Incident Report Service Recovery System Maintenance    ',
 'escalated': True,
 'escalate_It': True}

In [ ]:
# prompt: escalate issues for all of the rows

import gspread
from google.oauth2.service_account import Credentials
import pandas as pd

# ... (Your existing code for authentication and data loading)

def escalateit(incoming_issues):
    tags_combine = " ".join([incoming_issues.get(f"tag_{i+1}", "") for i in range(9)])

    # Check for high-priority and critical keywords
    critical_keywords = ["issue", "problem", "urgent", "disruption", "failure",
                         "incident", "crash", "refund", "outage", "critical"]
    if incoming_issues["priority"] == "high" and any(key.lower() in tags_combine.lower() for key in critical_keywords):
        return True

    # Escalate if specific tags are present
    specific_tags = ["security", "data breach", "compliance"]
    if any(tag.lower() in tags_combine.lower() for tag in specific_tags):
        return True

    return False

# Apply the function to each row of the DataFrame
df_sheets['escalate_It'] = df_sheets.apply(escalateit, axis=1)

# Display the updated DataFrame with the 'escalated' column
# print(df_sheets.head())

# Update the Google Sheet with the new 'escalated' column
# Convert the DataFrame back to a list of lists
updated_data = df_sheets.values.tolist()

# Insert the header row back into the data
updated_data.insert(0, df_sheets.columns.tolist())

# # Update the Google Sheet
# worksheet.update('A1', updated_data) # Update starting from cell A1

# print("Google Sheet updated successfully!")
df_sheets.head()

,subject,body,answer,type,queue,priority,language,business_type,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9,all_tags,escalate_It
0,Problema crítico del servidor requiere atenció...,Es necesaria una investigación inmediata sobre...,Estamos investigando urgentemente el problema ...,Incident,Technical Support,high,es,IT Services,Urgent Issue,Service Disruption,Incident Report,Service Recovery,System Maintenance,,,,,Urgent Issue Service Disruption Incident Repor...,True
1,Anfrage zur Verfügbarkeit des Dell XPS 13 9310,"Sehr geehrter Kundenservice,\n\nich hoffe, die...","Sehr geehrter <name>,\n\nvielen Dank, dass Sie...",Request,Customer Service,low,de,Tech Online Store,Sales Inquiry,Product Support,Customer Service,Order Issue,Returns and Exchanges,,,,,Sales Inquiry Product Support Customer Service...,False
2,Erro na Autocompletação de Código do IntelliJ ...,"Prezado Suporte ao Cliente <name>,\n\nEstou es...","Prezado <name>,\n\nObrigado por entrar em cont...",Incident,Technical Support,high,pt,IT Services,Technical Support,Software Bug,Problem Resolution,Urgent Issue,IT Support,,,,,Technical Support Software Bug Problem Resolut...,True
3,Urgent Assistance Required: AWS Service,"Dear IT Services Support Team, \n\nI am reachi...","Dear <name>,\n\nThank you for reaching out reg...",Request,IT Support,high,en,IT Services,IT Support,Urgent Issue,Service Notification,Cloud Services,Problem Resolution,Technical Guidance,Performance Tuning,,,IT Support Urgent Issue Service Notification C...,True
4,Problème d'affichage de MacBook Air,Cher équipe de support du magasin en ligne Tec...,"Cher <name>,\n\nMerci de nous avoir contactés ...",Incident,Product Support,low,fr,Tech Online Store,Technical Support,Product Support,Hardware Failure,Service Recovery,Routine Request,,,,,Technical Support Product Support Hardware Fai...,False
